In [1]:
import requests
import re
import pandas as pd
#look for python library for querying wikipedia


# ``` import wikipedia
# results = wikipedia.search(query='Machine Learning', results = 10)
# page = wikipedia.WikipediaPage( results[0], preload = True)
# page.content```

In [ ]:
!pip install psycopg2 --quiet

## CRUD

| | SQL | RESTful API |
|:-:|:-:|:-:|
| create | `INSERT` | `POST` |
| read | `SELECT` | `GET` |
| update | `UPDATE` | `PUT` |
| delete | `DELETE` | `DELETE` |

![](http://interactive.blockdiag.com/image?compression=deflate&encoding=base64&src=eJxLyslPzk7JTExXqOZSUFAvLE0tqlQoz8zOLEgFiqor6NopFKUWl-aUFIOY6uVFmSWpCiX5CgX5xSXpQBl1a65aACFZFx4)

![](http://interactive.blockdiag.com/image?compression=deflate&encoding=base64&src=eJxLyslPzk7JTExXqOZSUFAPcnV0UUgrys9VKM_MzixIBcqoK-jaKRSlFpfmlBSDmOqefsGuQSEKJfkKBfnFJelAKXVrrloAaBAXgg)

In [ ]:
response=requests.get("https://google.com")

### Query Wikipedia

In [ ]:
response_wiki=requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q39246&format=json")

In [ ]:
response_json= response_wiki.json()

In [ ]:
response_json.keys()

In [ ]:
base_url = "https://en.wikipedia.org/w/api.php"

In [ ]:
action = "?action=mobileview"
parameters = "&format=json&prop=sections&sections=all"
page = "&page="

In [ ]:
my_response=requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&explaintext=True&cmlimit=max&cmtitle=Category:Machine_learning")





In [ ]:
my_response=my_response.json()

In [ ]:
my_response["query"]["categorymembers"][:5]

In [ ]:
            http://en.wikipedia.org/w/api.php?
            action=query&
            format=json&
            list=categorymembers&
            cmtitle=Category:{}& 
            cmlimit=max

In [ ]:
    https://en.wikipedia.org/w/api.php?
    action=query&
    format=json&
    list=categorymembers&
    explaintext=True&cmlimit=max&cmtitle=Category:Machine_learning

In [ ]:
def get_page_ids(category):
    if category[:9] == "Category:":
        category= category[9:]
    #replace spaces with underscore for articles longer than 1 word
    category.replace(" ", "_")    
        
    base_url="http://en.wikipedia.org/w/api.php"
    action="?action=query"
    parameters="&format=json&list=categorymembers&explaintext=True&cmlimit=max&cmtitle=Category:"
    
    response=requests.get(base_url+action+parameters+category)
    json=response.json()
    articles=json["query"]["categorymembers"]
    

    
    page_ids=[]
    title=[]
    
    for i in range(len(articles)):
        #base case
        if articles[i]["ns"]!=14:
            page_ids.append(articles[i]["pageid"])
            title.append(articles[i]["title"])
        #recursion to get subcategories
        elif articles[i]["ns"]==14:
            page_ids += get_page_ids(articles[i]["title"])
        
    
    return dict(zip(page_ids, title))

In [ ]:
my_articles=get_page_ids("Machine_learning")

In [ ]:
len(my_articles)

In [ ]:
text=[]
count=0
for i in my_articles.keys():
    try:
        text.append(get_text(i))
        count+=1
    except:
        print(count)
text

In [ ]:
!sudo service postgresql start

## 

In [ ]:
#Startttt

In [ ]:
!pip install wikipedia

In [ ]:
get_text(44439173)[:2000]

In [3]:
import wikipedia
def get_text(page_id):
    page = wikipedia.WikipediaPage(pageid=str(page_id), preload = True)
    return cleaner(page.content)

In [4]:
def cleaner(message):
    message = re.sub('\.+', ' ', message)
    message = re.sub('[^a-z0-9 ]','', message.lower())
    message = re.sub('\d+','NUMBER ',message)
    message = re.sub('\s+',' ',message)
    return message

In [ ]:
import wikipedia
# results = wikipedia.search(query="M", results = 10)
page = wikipedia.WikipediaPage(pageid="44439173", preload = True)
page.content

In [ ]:
PART 1 todo:
    -make schema
    -pull from wikipedia in a usable form (dictionary) to push to postgresSQL
    

In [ ]:
article=wikipedia.page('"Hello, World!" program')
article.content

In [5]:
def generate_category(category):
    '''
    format a category for insertion in to a wikipedia api call
    '''
    category = re.sub('\s','_',category)
    return category

In [ ]:
generate_category('Machine learning')

In [6]:
def generate_query(category):
    '''
    Format an api call for requests
    '''
    query = """
            http://en.wikipedia.org/w/api.php?
            action=query&
            format=json&
            list=categorymembers&
            cmlimit=max&
            explaintext=True&
            cmtitle=Category:{} 
            
            """.format(generate_category(category))
    query = re.sub('\s','',query)
    return query

In [ ]:
generate_query('Business_software')

In [7]:
def execute_category_query(category):
    '''
    Executes a category qeury and returns a 
    DataFrame of the category members
    '''
    
    r = requests.get(generate_query(category))
    response = r.json()
    return pd.DataFrame(response['query']['categorymembers'])

In [ ]:
execute_category_query('Business software')

In [8]:
def remove_category(category):
    category = re.sub('Category:','',category)
    return category

In [ ]:
remove_category('machine learning')

In [ ]:
categories_to_query = test[category_mask]['title'].apply(remove_category).tolist()

In [16]:
def get_all_pages_rec(category, level):
    category_df = execute_category_query(category)
    
    if category_df.shape == (0, 0):
        return None
    pages_list = []
    
    
    category_mask = category_df['title'].str.contains('Category:')
    pages_df = category_df[~category_mask]
    pages_list.append(pages_df)
    
    categories = category_df[category_mask]['title']\
                            .str.replace('Category:','').tolist()
    if len(categories) > 0:
        for cat in categories:
            if level>0:
                level-=1
                pages_list.append(get_all_pages_rec(cat, level-1))
                level+=1
    pages_df = pd.concat(pages_list)
    pages_df.reset_index(drop=True)
    return pages_df

In [22]:
def get_whole_category(category):
    df = get_all_pages_rec(category, 5)
    df = df.drop_duplicates().reset_index(drop=True)
    df['category_id'] = category
    return df

In [23]:
ml_df=get_whole_category('Machine learning')
ml_df.shape

(1107, 4)

In [19]:
bs_df=get_whole_category('Business software')
bs_df.shape

(3054, 4)

In [ ]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor

In [ ]:
connection = pg2.connect(host="postgres", database="wiki", user="postgres")
cursor = connection.cursor(cursor_factory=RealDictCursor)
#sqlalchemy

In [ ]:
bs_df.head()

In [ ]:
def insert_table(pageid, title, category_id, engine):
    stmt=("INSERT {} INTO wiki".format(pageid, title, category_id)
    
    return engine.execute(stmt)

## Writing to Postgres / PostgresSQL schema

We will want the following two tables:
    page
    ====
    title
    page_ID
    text
    
    subheadings
    notes
    
    
    category
    ========
    category_ID
    title
    
    
    page-category
    ============
    page_category_id
    page_id
    category_id
    
    

|page_id | title |
|:-:|:-:|
| 1 | logistic regression |
| 2 | perceptron | 
| 3 | random forest | 

| category_id | title|
|:-:|:-:|
| 1 | machine learning |
| 2 | linear models |
| 3 | tree models |


|p_c_id | p_id | c_id|
|:-:|:-:|:-:|
| 1 | 1 | 1 |
| 2 | 1 | 2 |
| 3 | 2 | 1 |
| 4 | 2 | 2 |
| 5 | 3 | 1 |
| 6 | 3 | 3 |

In [ ]:
#unresolved:  2) put all text in database 3) TYPES OF tables put in using to_sql

# # To do
# 1) LSA- search query find top 5 articles lsa->prediction
#    -query database fore text; vectorize, SVD
# 2) PREDICT CATEGORY OF TERM -mikes -> description
